# Day 1 - Lab 1: AI-Powered Requirements & User Stories

**Objective:** Use a Large Language Model (LLM) to decompose a vague problem statement into structured features, user personas, and Agile user stories, culminating in a machine-readable JSON artifact.

**Estimated Time:** 90 minutes

**Introduction:**
Welcome to the first hands-on lab of the AI-Driven Software Engineering Program! All great software projects begin with a clear understanding of the problem to be solved. In this lab, you will take on the role of a tech lead or product manager and use an LLM as a co-pilot to transform a simple, high-level problem into a set of well-defined, actionable requirements. This process is fundamental to ensuring that the team builds the *right* product.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

This initial block sets up our environment. It adds the project's root directory to the Python path, allowing us to import our custom `utils.py` script. We then initialize the connection to our Large Language Model (LLM).

**Model Selection:**
Our `utils.py` script is configured to work with multiple AI providers. You can change the `model_name` parameter in the `setup_llm_client()` function to any of the models listed in the `RECOMMENDED_MODELS` dictionary in `utils.py`. For example, to use a Hugging Face model, you could change the line to: `client, model_name, api_provider = setup_llm_client(model_name="meta-llama/Llama-3.3-70B-Instruct")`

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client for our chosen LLM.
- `get_completion()`: To send a prompt to the LLM and get a response.
- `save_artifact()`: To save our generated requirements to a file.

In [44]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact

# Initialize the LLM client. You can change the model here.
# For example: setup_llm_client(model_name="gemini-2.5-flash")
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

2025-09-22 16:10:31,467 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


In [45]:
def add_numbers(a, b):
    """Returns the sum of two numbers."""
    return a + b

## Step 2: The Problem Statement

Every project starts with a problem. Our problem is a common one in many organizations:

> **"We need a tool to help our company's new hires get up to speed."**

This statement is intentionally vague. Our job is to use the LLM to add structure and detail to it.

In [46]:
problem_statement = "We need a tool to help our company's new hires get up to speed."

## Step 3: The Challenges

Complete the following challenges in order. Each one builds upon the last, increasing in technical complexity and value.

### Challenge 1 (Foundational): Brainstorming Features

**Task:** Use the LLM to brainstorm a list of potential features and user personas based on the problem statement.

**Instructions:**
1. Write a simple prompt that asks the LLM to brainstorm features for the onboarding tool.
2. Write a second prompt to identify three distinct user personas who would use this tool.
3. Run both prompts and review the markdown output.

**Expected Quality:** The output should be a simple, readable markdown list of features and a description of the personas. This is a good first step but lacks the structure needed for automation.

In [47]:
# TODO: Create a string variable named 'features_prompt'.
# This prompt should ask the LLM to list features for the problem_statement.
features_prompt = f"""
List features that would help solve the following problem.

Problem statement:
{problem_statement}
"""

print("--- Brainstorming Features ---")
brainstormed_features = get_completion(features_prompt, client, model_name, api_provider)
print(brainstormed_features)

# TODO: Create a string variable named 'personas_prompt'.
# This prompt should ask the LLM to identify three user personas for the problem_statement.
personas_prompt = f"""
Identify three user personas who would use a tool designed for the following problem.

Problem statement:
{problem_statement}
"""

print("\n--- Identifying User Personas ---")
user_personas = get_completion(personas_prompt, client, model_name, api_provider)
print(user_personas)

--- Brainstorming Features ---
To address the problem of helping new hires get up to speed, consider developing a tool with the following features:

1. **Onboarding Checklist**: A comprehensive list of tasks and goals for new hires to complete during their onboarding period.

2. **Interactive Orientation Modules**: Engaging and self-paced modules covering company culture, values, policies, and procedures.

3. **Role-Specific Training**: Tailored content and resources relevant to the specific roles of new employees.

4. **Mentorship Matching**: Pairing new hires with experienced employees who can provide guidance and support.

5. **Knowledge Base Access**: A centralized repository of company documents, FAQs, and training materials that is easily searchable.

6. **Progress Tracking**: A feature allowing new hires and their managers to track completion of onboarding tasks and training modules.

7. **Feedback Mechanism**: An easy way for new hires to provide feedback on the onboarding proc

### Challenge 2 (Intermediate): Generating Formal User Stories

**Task:** Now, let's increase the value by generating structured, formal Agile User Stories.

**Instructions:**
1. Create a new, more sophisticated prompt.
2. This prompt should instruct the LLM to act as a Senior Product Manager.
3. It must use the brainstormed features and personas from the previous step as context.
4. The key instruction is to generate a list of user stories, each with detailed acceptance criteria in Gherkin format (`Given/When/Then`).
5. **Crucially, the prompt must demand the final output be a well-formed JSON array of objects.** Each object should represent a user story and have keys like `id`, `user_story`, `persona`, and `acceptance_criteria`.

> **Tip:** If the LLM's output isn't perfect JSON, try making your prompt even more specific. You can tell it, 'Do not include any text before or after the JSON array. Your response must begin with [ and end with ].'

**Expected Quality:** The output should not be markdown, but a clean, parsable JSON string. This is a significant step up in value, as a JSON artifact can be automatically processed by other systems (e.g., imported into Jira).

In [48]:
# TODO: Create a detailed prompt string named 'json_user_stories_prompt'.
# This prompt needs to instruct the LLM to act as a Senior Product Manager and convert the
# brainstormed features and personas into a structured JSON array of user stories.
# Tip: Be very specific about the required JSON format in your prompt instructions. Tell it what keys to use and what the data types should be.
json_user_stories_prompt = f"""
You are a Senior Product Manager.
Convert the following features and user personas into a structured JSON array of user stories.

Instructions:
- Output ONLY a JSON array, with no extra text or markdown fences.
- Each user story must be a JSON object with the following keys:
  - "id": integer (unique for each story)
  - "user_story": string (Agile user story format)
  - "persona": string (one of the provided personas)
  - "acceptance_criteria": list of strings (each in Gherkin format: Given/When/Then)
- The response must begin with [ and end with ].

Features:
{brainstormed_features}

User Personas:
{user_personas}
"""

print("--- Generating User Stories as JSON ---")
json_output_str = get_completion(json_user_stories_prompt, client, model_name, api_provider, temperature=0.2)

# Let's try to parse the JSON to see if the LLM followed instructions
import re
try:
    # Remove markdown fences if present
    if '```' in json_output_str:
        # Extract content between fences
        match = re.search(r"```(?:json)?\s*(.*?)```", json_output_str, re.DOTALL)
        if match:
            json_output_str = match.group(1).strip()
    # Remove any leading/trailing whitespace
    json_output_str = json_output_str.strip()
    # Try to parse
    user_stories_json = json.loads(json_output_str)
    print("Successfully parsed LLM output as JSON.")
    if user_stories_json:
        print("\n--- All User Stories ---")
        print(json.dumps(user_stories_json, indent=2))
    else:
        print("JSON array is empty.")
except (json.JSONDecodeError, TypeError, IndexError) as e:
    print(f"Error: Failed to parse LLM output as JSON. Error: {e}")
    print("LLM Output was:\n", json_output_str)
    user_stories_json = []

--- Generating User Stories as JSON ---
Successfully parsed LLM output as JSON.

--- All User Stories ---
[
  {
    "id": 1,
    "user_story": "As a Human Resources Manager, I want a comprehensive onboarding checklist so that new hires can complete tasks and goals efficiently during their onboarding period.",
    "persona": "Sarah",
    "acceptance_criteria": [
      "Given a new hire starts onboarding, when they access the checklist, then they should see a list of tasks and goals to complete.",
      "Given a task is completed, when the new hire marks it as done, then it should be reflected in the checklist."
    ]
  },
  {
    "id": 2,
    "user_story": "As a New Hire, I want interactive orientation modules so that I can learn about the company culture, values, policies, and procedures at my own pace.",
    "persona": "Emily",
    "acceptance_criteria": [
      "Given I start an orientation module, when I complete a section, then I should be able to proceed to the next section.",
   

### Challenge 3 (Advanced): Programmatic Validation and Artifact Creation

**Task:** Now for the highest-value step. Instead of just looking at the JSON, we will programmatically validate it and save it as a formal project artifact. This ensures reliability and prepares the requirements for automated use in later stages of the SDLC.

**Instructions:**
1. Complete the `validate_and_save_stories` function below.
2. The function should iterate through the list of stories.
3. For each story, it must validate that the required keys are present and that the acceptance criteria list is not empty.
4. If all stories are valid, it should save the data to `artifacts/day1_user_stories.json`.

**Expected Quality:** A robust script that guarantees the integrity of our requirements artifact. The final output is a validated `day1_user_stories.json` file in the `artifacts` directory, ready to be used as a reliable input for Day 2.

In [49]:
def validate_and_save_stories(stories_data):
    """Validates the structure of the user stories data and saves it if valid."""
    if not isinstance(stories_data, list) or not stories_data:
        print("Validation Failed: Data is not a non-empty list.")
        return

    required_keys = ['id', 'persona', 'user_story', 'acceptance_criteria']
    all_stories_valid = True

    for story in stories_data:
        if not isinstance(story, dict):
            print(f"Validation Failed: Story is not a dictionary: {story}")
            all_stories_valid = False
            continue
        
        missing_keys = [key for key in required_keys if key not in story]
        if missing_keys:
            print(f"Validation Failed: Story is missing keys {missing_keys}: {story}")
            all_stories_valid = False
            continue
        
        if not isinstance(story['acceptance_criteria'], list) or not story['acceptance_criteria']:
            print(f"Validation Failed: 'acceptance_criteria' is not a non-empty list in story: {story}")
            all_stories_valid = False
            continue

    if all_stories_valid:
        print("\nAll user stories passed validation.")
        artifact_path = "artifacts/day1_user_stories.json"
        
        # Save the validated stories as a JSON artifact
        from utils import save_artifact
        import json
        save_artifact(json.dumps(stories_data, indent=2), artifact_path)
        print(f"Artifact saved to {artifact_path}")
    else:
        print("\nValidation failed. Artifact not saved.")

# Run the validation on the JSON data from the previous step
if 'user_stories_json' in locals() and user_stories_json:
    validate_and_save_stories(user_stories_json)
else:
    print("Skipping validation as user_stories_json is empty or not defined.")


All user stories passed validation.
Artifact saved to artifacts/day1_user_stories.json


## Lab Conclusion

Congratulations! You have completed the first lab. You started with a vague, one-sentence problem and finished with a structured, validated, machine-readable requirements artifact. This is the critical first step in an AI-assisted software development lifecycle. The `day1_user_stories.json` file you created will be the direct input for our next lab, where we will generate a formal Product Requirements Document (PRD).

> **Key Takeaway:** The single most important skill demonstrated in this lab is turning unstructured ideas into structured, machine-readable data (JSON). This transformation is what enables automation and integration with other tools later in the SDLC.